In [1]:
import cv2 
import numpy as np 
import imutils

img=cv2.imread('hand1.jpeg',1)
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur=cv2.GaussianBlur(gray,(3,3),0)


threash=cv2.threshold(blur,120,255,cv2.THRESH_BINARY)[1]
threash=cv2.erode(threash,None,iterations=2)
threash=cv2.dilate(threash,None,iterations=4)


cnts=cv2.findContours(threash,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)
c=max(cnts,key=cv2.contourArea)

# determine the most extreme points along the contour
extLeft = tuple(c[c[:, :, 0].argmin()][0])
extRight = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])




cv2.drawContours(img, [c], -1, (0, 255, 255), 2)
cv2.circle(img, extLeft, 8, (0, 0, 255), -1)
cv2.circle(img, extRight, 8, (0, 255, 0), -1)
cv2.circle(img, extTop, 8, (255, 0, 0), -1)
cv2.circle(img, extBot, 8, (255, 255, 0), -1)
cv2.imshow('hand',threash)
cv2.imshow('img',img)
cv2.waitKey(0)&0xFF
cv2.destroyAllWindows()


In [37]:
c[:, :, 0].argmin()


523

In [39]:
c[c[:, :, 0].argmin()]

array([[ 12, 384]], dtype=int32)

In [2]:
#HSV hue saturationn value
import cv2 
import numpy as np 

frame=cv2.imread('./bottles/all bottle 2.jpg',1)
hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
l_b=np.array([12,14,74])#lower bound
u_b=np.array([79,255,255])#upper bound
    
mask=cv2.inRange(hsv,l_b,u_b)
#res=cv2.bitwise_and(frame,frame,mask=mask) #extrac the object from the image
#threash=cv2.erode(mask,None,iterations=1)
threash=cv2.dilate(mask,None,iterations=2)

cnts=cv2.findContours(threash,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)
c=max(cnts,key=cv2.contourArea)


# determine the most extreme points along the contour
extLeft = tuple(c[c[:, :, 0].argmin()][0])
extRight = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])




cv2.drawContours(frame, [c], -1, (0, 255, 255), 2)
cv2.circle(frame, extLeft, 8, (0, 0, 255), -1)
cv2.circle(frame, extRight, 8, (0, 255, 0), -1)
cv2.circle(frame, extTop, 8, (255, 0, 0), -1)
cv2.circle(frame, extBot, 8, (255, 255, 0), -1)

cv2.imshow('frame',frame)
cv2.imshow('mask',mask)
cv2.imshow('threash',threash)
cv2.waitKey(0)& 0xFF ==27

cv2.destroyAllWindows()
    

In [7]:
"""
Filename: init.py
Usage: This script will measure different objects in the frame using a reference object of known dimension. 
The object with known dimension must be the leftmost object.
Author: Shashank Sharma
"""
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

# Function to show array of images (intermediate results)


def show_images(images):
    for i, img in enumerate(images):
        cv2.imshow("image_" + str(i), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


#img_path = "G:/coding_files/openCV/github code for size/object-size/images/example_02.jpg"

img_path = "G:/coding_files/openCV/bottle detection/bottles/all bottles.jpeg"


# Read image and preprocess
image = cv2.imread(img_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9, 9), 0)

edged = cv2.Canny(blur, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

#show_images([blur, edged])

# Find contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# Sort contours from left to right as leftmost contour is reference object
(cnts, _) = contours.sort_contours(cnts)

# Remove contours which are not large enough
cnts = [x for x in cnts if cv2.contourArea(x) > 100]

#cv2.drawContours(image, cnts, -1, (0,255,0), 3)

#show_images([image, edged])
# print(len(cnts))

# Reference object dimensions
# Here for reference I have used a 2cm x 2cm square
ref_object = cnts[0]
box = cv2.minAreaRect(ref_object)
box = cv2.boxPoints(box)
box = np.array(box, dtype="int")
box = perspective.order_points(box)
(tl, tr, br, bl) = box
dist_in_pixel = euclidean(tl, tr)
dist_in_cm = 1
pixel_per_cm = dist_in_pixel/dist_in_cm

# Draw remaining contours
for cnt in cnts:
    box = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box
    cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2),
                         tl[1] + int(abs(tr[1] - tl[1])/2))
    mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2),
                       tr[1] + int(abs(tr[1] - br[1])/2))
    wid = euclidean(tl, tr)/pixel_per_cm
    ht = euclidean(tr, br)/pixel_per_cm
    cv2.putText(image, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(image, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

show_images([image])

In [4]:
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

# Function to show array of images (intermediate results)


def show_images(images):
    for i, img in enumerate(images):
        cv2.imshow("image_" + str(i), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


img_path = "G:/coding_files/openCV/bottle detection/bottles/all bottles.jpeg"

# Read image and preprocess
image = cv2.imread(img_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9, 9), 0)

edged = cv2.Canny(blur, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)


cv2.imshow('edged',edged)
cv2.waitKey(0)&0xFF==27
cv2.destroyAllWindows( )



In [3]:
#HSV hue saturationn value
import cv2 
import numpy as np 
import imutils
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
frame=cv2.imread('./bottles/small bottle g1.jpg',1)
frame=cv2.resize(frame, (1280 ,768 )) 

hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
# l_b=np.array([12,14,74])#lower bound
# u_b=np.array([79,255,255])#upper bound
    
l_b=np.array([0,88,65])#lower bo und
u_b=np.array([255,255,255 ])#upper bound    
mask=cv2.inRange(hsv,l_b,u_b)
#res=cv2.bitwise_and(frame,frame,mask=mask) #extrac the object from the image
#threash=cv2.erode(mask,None,iterations=1)
threash=cv2.dilate(mask,None,iterations=2)

cnts=cv2.findContours(threash,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)

(cnts, _) = contours.sort_contours(cnts)

# Remove contours which are not large enough
cnts = [x for x in cnts if cv2.contourArea(x) > 100]

ref_object = cnts[0]
box = cv2.minAreaRect(ref_object)
box = cv2.boxPoints(box)
box = np.array(box, dtype="int")
box = perspective.order_points(box)
(tl, tr, br, bl) = box
dist_in_pixel = euclidean(tl, tr)
dist_in_cm = 2
pixel_per_cm = dist_in_pixel/dist_in_cm
length=[]
# Draw remaining contours
for cnt in cnts:
    box = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box
    cv2.drawContours(frame, [box.astype("int")], -1, (0, 0, 255), 2)
    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2),
                         tl[1] + int(abs(tr[1] - tl[1])/2))
    mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2),
                       tr[1] + int(abs(tr[1] - br[1])/2))
    wid = euclidean(tl, tr)/pixel_per_cm
    ht = euclidean(tr, br)/pixel_per_cm
    length.append([wid,ht])
    cv2.putText(frame, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(frame, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)



cv2.imshow('frame',frame)
cv2.imshow('mask',mask)
cv2.imshow('threash',threash)
cv2.waitKey(0)& 0xFF ==27

cv2.destroyAllWindows()
print(length)

[[2.0, 1.5250858263691143], [5.1245997383894215, 9.324271719118313], [13.323959319812495, 16.123740640298422]]
